**정확도 1.0 나온 모델 개선**

데이터 처리 수정: 음이 빈 데이터를 -1로 채워진 벡터를 추가.

모델 학습 후 혼동 행렬 시각화: 모델 예측 결과를 바탕으로 혼동 행렬을 시각화.

모델 저장 기능 추가: 학습이 완료된 모델을 로컬 파일로 저장.

In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
df = pd.read_csv('/content/drive/My Drive/soundAI/df_concat.csv')

# 문자열을 실제 텐서로 변환하는 함수
def convert_to_tensor(tensor_str):
    tensor_list = ast.literal_eval(tensor_str)
    tensor = torch.tensor(tensor_list)
    return tensor

df['feature'] = df['feature'].apply(convert_to_tensor)

# 노이즈 추가 함수 정의
def add_noise(tensor, noise_level=0.01):
    noise = np.random.normal(0, noise_level, tensor.shape)
    return tensor + noise

# 시간 축 이동 함수 정의
def time_shift(tensor, shift_max=2):
    shift = np.random.randint(-shift_max, shift_max)
    return np.roll(tensor, shift, axis=1)

# 시간 축 스케일링 함수 정의
def time_stretch(tensor, stretch_factor=0.2):
    length = tensor.shape[1]
    stretched_length = int(length * (1 + stretch_factor))
    stretched_tensor = np.zeros((tensor.shape[0], stretched_length))
    for i in range(tensor.shape[0]):
        stretched_tensor[i] = np.interp(np.linspace(0, length, stretched_length), np.arange(length), tensor[i])
    return stretched_tensor

# 데이터 증강 함수
def augment_data(df, num_augmentations=10, noise_level=0.01):
    augmented_features = []
    augmented_labels = []

    for _, row in df.iterrows():
        feature = np.array(row['feature'])
        label = row['label']

        augmented_features.append(feature)
        augmented_labels.append(label)

        for _ in range(num_augmentations - 1):
            augmented_feature = add_noise(feature, noise_level)
            augmented_feature = time_shift(augmented_feature)
            augmented_feature = time_stretch(augmented_feature)
            augmented_features.append(augmented_feature)
            augmented_labels.append(label)

    augmented_df = pd.DataFrame({
        'feature': augmented_features,
        'label': augmented_labels
    })
    return augmented_df

# 데이터 증강 실행
augmented_df = augment_data(df)

# 패딩 함수 정의
def pad_feature(feature, target_shape=(1025, 130), pad_value=-1):
    current_shape = feature.shape
    padded_feature = np.full(target_shape, pad_value)
    if current_shape[1] > target_shape[1]:
        padded_feature[:, :] = feature[:, :target_shape[1]]
    else:
        padded_feature[:, :current_shape[1]] = feature
    return padded_feature

# 빈 데이터를 -1로 채운 벡터 추가
def add_empty_data(df, pad_value=-1, target_shape=(1025, 130)):
    empty_feature = np.full(target_shape, pad_value)
    empty_data = pd.DataFrame({
        'feature': [empty_feature for _ in range(df['feature'].shape[0])],
        'label': [-1 for _ in range(df['feature'].shape[0])]
    })
    df = pd.concat([df, empty_data], ignore_index=True)
    return df

# 데이터 프레임에 빈 데이터 추가
df = add_empty_data(df)
augmented_df = add_empty_data(augmented_df)

# 각 행마다 패딩 적용
augmented_df['feature'] = augmented_df['feature'].apply(lambda x: pad_feature(np.array(x)))
df['feature'] = df['feature'].apply(lambda x: pad_feature(np.array(x)))

# 레이블 매핑
unique_labels = np.sort(augmented_df['label'].unique())
label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}

augmented_df['label'] = augmented_df['label'].map(label_mapping)
df['label'] = df['label'].map(label_mapping)

# 데이터 및 레이블 준비
X_train = np.stack(augmented_df['feature'].values)
y_train = augmented_df['label'].values

X_test = np.stack(df['feature'].values)
y_test = df['label'].values

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# 2D CNN 모델 정의
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1025, 130, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

model.summary()

history = model.fit(X_train, y_train_categorical, validation_data=(X_test, y_test_categorical), epochs=100, batch_size=64, callbacks=[reduce_lr])

loss, accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy: {accuracy}')

# 정확도와 손실 값을 시각화하는 함수
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy over Epochs')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss over Epochs')

    plt.show()

plot_history(history)

# 혼동 행렬 시각화
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1023, 128, 32)     320       
                                                                 
 batch_normalization (Batch  (None, 1023, 128, 32)     128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 511, 64, 32)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 511, 64, 32)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 509, 62, 64)       18496     
                                                                 
 batch_normalization_1 (Bat  (None, 509, 62, 64)       2

In [ ]:
# 모델 저장
model.save('/content/drive/My Drive/soundAI/model.h5')